# Data Preparation

### Counts data

In [ ]:
from moonstone.parsers.counts.taxonomy import SunbeamKraken2Parser


krakenfile = "moonstone_tuto_kraken2_file.tsv"
parser = SunbeamKraken2Parser(krakenfile)
counts_dataframe = parser.dataframe.drop('NCBI_taxonomy_ID', axis=1)

In [ ]:
counts_dataframe

In [ ]:
counts_dataframe.sum()

#### Normalization of data

SAMPLE_3 number of classified reads is 8 to 24 times greater than other samples' reads counts. So that futur analyses not to be biased towards SAMPLE_3, we need to perform some kind of normalization.

Moonstone offers assistance in normalizing your data using many different methods (see [list](https://moonstone.readthedocs.io/en/latest/api_docs/normalization.html))

(To better understand what each normalization method entails, you can watch this [youtube video](https://www.youtube.com/watch?v=UFB993xufUU&t=683s))

In [ ]:
from moonstone.normalization.counts.geometric_mean import (
    GeometricMeanNormalization
)

geom_mean_norm = GeometricMeanNormalization(counts_dataframe)     # instantiation
counts_dataframe_normalized = geom_mean_norm.normalized_df

In [ ]:
counts_dataframe_normalized.sum()

### Metadata

In [ ]:
import pandas as pd

metadata_file = "moonstone_tuto_metadata_file.csv"
metadata_dataframe = pd.read_csv(metadata_file, sep=",", index_col="SAMPLE_ID")

In [ ]:
metadata_dataframe

In [ ]:
pd.crosstab(metadata_dataframe['SMOKER'], metadata_dataframe['GROUP'])

# Data visualization/exploration

In [ ]:
from moonstone.plot.counts import PlotTaxonomyCounts

instance = PlotTaxonomyCounts(counts_dataframe_normalized)

In [ ]:
fig1 = instance.plot_most_prevalent_taxa(
    mode="bar",
    mean_threshold=6,
    taxa_number=3,
    taxa_level="species",
    higher_classification=False,               # Set to False remove every rows "xxx(higher taxa)"
    ascending=False,
)

In [ ]:
fig2 = instance.plot_most_abundant_taxa(
    mode="boxplot",
    taxa_level="species",
    prevalence_threshold=None,
    average_relative_abundance_threshold=5,
    higher_classification=False,                
    ascending=False,
    output_file="Most_abundant_species.html"        # It's also possible to generate a static image. You just need to change the extension of the file
)

In [ ]:
fig3 = instance.plot_sample_composition_most_abundant_taxa(
    taxa_level="species",
    taxa_number=3,                        # the X top species will be represented, the other will be under "Others"
    cluster_samples=True,                 # cluster samples according to their composition in top species/Others (default set to True)
    colors={"Others" : "#d1dae8"},        # set the color of a species (or of "Others")
    color_df=metadata_dataframe["GROUP"], # series or dataframe of metadata to add at the bottom of the graph
    sep_series=metadata_dataframe["SMOKER"].replace({"yes": "smoker", "no": "non smoker"}),
    sep_how="labels",
    output_file=None,
    plotting_options={"layout": {"yaxis_title": "Relative abundance"}}   # all moonstone's plot and graph methods allow you to give your own plotting options
                                                                         # it relies on the fig.update_X({*dictionary*}) methods of plotly
                                                                         # it should be given in a dictionary with X being the first key and then a dictionary of the
                                                                         # parameters to update as you would give the fig.update_X method
)

# Diversity analysis

The Alpha diversity (α-diversity) corresponds to the intra-sample diversity, whereas the Beta diversity (β-diversity) corresponds to the inter-samples diversity.

In [ ]:
"""
Alpha-Diversity (at species-level)
"""

from moonstone.analysis.diversity.alpha import ShannonIndex

cdn_species = counts_dataframe_normalized.groupby("species").sum()

alpha_div_instance = ShannonIndex(cdn_species)
alpha_div_instance.diversity_indexes.to_csv(f"ShannonIndex.csv")
grouped_output = alpha_div_instance.analyse_groups(
    metadata_df=metadata_dataframe, group_col="SMOKER",
    stats_test="mann_whitney_u", correction_method="fdr_bh",
    boxpoints="suspectedoutliers", structure_pval="series", sym=False,
    show_pval=False,
)
print(grouped_output["pval"])

In [ ]:
"""
Beta-Diversity (at species-level)
"""

from moonstone.analysis.diversity.beta import BrayCurtis

cdn_species = counts_dataframe_normalized.groupby("species").sum()

beta_div_instance = BrayCurtis(cdn_species)
beta_div_instance.diversity_indexes.to_csv(f"BrayCurtis.csv")
grouped_output = beta_div_instance.analyse_groups(
    metadata_df=metadata_dataframe, group_col="SMOKER", group_col2="GROUP",
    stats_test="mann_whitney_u", correction_method="fdr_bh",
    boxpoints="suspectedoutliers", structure_pval="series", sym=False,
    show_pval=False,
)
print(grouped_output["pval"])

# Other

In [ ]:
import pandas as pd

sp1 = [1.32, 44.67, 42.66, 28.18, 173.78, 912.01, 8.32, 11.22, 89.13, 5.01, 70.79, 12.3, 69.18, 5.01, 1.7, 107.15, 1.51, 204.17, 3.98, 21.38, 17.38, 38.9, 29.51, 575.44, 23.44, 204.17, 1.41, 398.11, 100.0, 1.05, 72.44, 3.16, 100.0, 9.12, 39.81, 15.49, 114.82, 724.44, 323.59, 1.35, 724.44, 112.2, 1.2, 30.9, 2.82, 6.17, 23.99, 6.46, 338.84, 309.03, 1.23, 2.45, 5.62, 40.74, 79.43, 602.56, 776.25, 7.59, 6.03, 302.0, 43.65, 446.68, 346.74, 3.39, 380.19, 870.96, 50.12, 6.03, 12.59, 309.03, 239.88, 120.23, 11.48, 2.0, 446.68, 28.18, 38.02, 8.51, 218.78, 2.57, 389.05, 645.65, 57.54, 18.62, 537.03, 562.34, 51.29, 123.03, 10.96, 1.17, 331.13, 1.95, 5.13, 457.09, 812.83, 64.57, 117.49, 85.11, 288.4, 257.04, 1]
sp2 = [50.28, 70.91, 72.02, 69.95, 79.66, 90.27, 60.47, 74.25, 76.92, 63.94, 75.72, 70.09, 64.55, 54.03, 59.07, 66.99, 54.3, 72.9, 57.11, 68.31, 74.85, 68.75, 68.42, 79.58, 68.88, 74.87, 60.0, 75.61, 73.05, 46.16, 74.56, 59.09, 78.65, 60.94, 62.9, 70.14, 64.51, 88.72, 88.07, 51.22, 83.69, 71.31, 59.41, 74.25, 52.11, 57.76, 66.57, 54.29, 81.95, 80.2, 58.05, 56.49, 57.55, 71.94, 76.22, 77.59, 84.46, 67.27, 62.08, 81.52, 40.07, 45.93, 47.8, 18.44, 39.25, 48.35, 36.25, 32.66, 21.75, 44.32, 40.31, 34.14, 24.54, 11.35, 39.98, 27.89, 30.53, 24.07, 39.09, 15.24, 47.14, 51.54, 33.11, 20.64, 42.57, 38.45, 27.63, 37.44, 31.59, 14.37, 31.81, 22.98, 22.42, 48.34, 46.34, 32.67, 42.26, 39.31, 39.78, 34.16, 100]
samples = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S42', 'S43', 'S44', 'S45', 'S46', 'S47', 'S48', 'S49', 'S50', 'S51', 'S52', 'S53', 'S54', 'S55', 'S56', 'S57', 'S58', 'S59', 'S60', 'S61', 'S62', 'S63', 'S64', 'S65', 'S66', 'S67', 'S68', 'S69', 'S70', 'S71', 'S72', 'S73', 'S74', 'S75', 'S76', 'S77', 'S78', 'S79', 'S80', 'S81', 'S82', 'S83', 'S84', 'S85', 'S86', 'S87', 'S88', 'S89', 'S90', 'S91', 'S92', 'S93', 'S94', 'S95', 'S96', 'S97', 'S98', 'S99', 'S100', 'outlier']
counts_dataframe2 = pd.DataFrame(
    [sp1, sp2], columns=samples, index=["species1", "species2"]
).T

In [ ]:
#from moonstone.plot.graphs import ScatterGraph
import importlib
import moonstone.plot.graphs.scatter as scatter
importlib.reload(scatter)

ins = scatter.ScatterGraph(counts_dataframe2)
f = ins.plot_one_graph("species1", "species2", plotting_options={"xaxes": {"type": "log"}})

The distribution seems to follow two trendlines. To investigate we want to divide samples into two groups, one following each trendlines

In [ ]:
from moonstone.plot.graphs import ScatterTrendlines

ins = ScatterTrendlines(counts_dataframe2)
output = ins.bootstraps_define_n_trendlines(
    "species1", "species2", 2, log_x=True, log_y=False, nb_iter=100, nb_bootstraps=5, outliers="trim",
    show=True
)
print(output["group_series"])
print(output["list_outliers"])